<a href="https://colab.research.google.com/github/gcosma/COP509/blob/main/LabSolutions/Lab_Exercise_(No_4a).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Lab Exercise (No. 4a)**

**Task Overview**

1. Your task is to use the cleaned ArtReviews dataset and the ArtRatings(classes) and repeat the Lab sheet to apply LSA.

2. Write code to retrieve the top 10 results for the 2 given queries.
['I really enjoy these scissors!',
'I hate the pen!'

###***pre-work1: mount drive***

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


###***pre-work2: set datasets***

In [ ]:
data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
!ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"

datasets = 'Arts5000' # ArtsAll, Arts5000


if datasets == 'ArtsAll':
  # Arts all
  data_all_reviews = 'ArtsReviews.txt'
  data_all_ratings = 'ArtsRatings.txt'
  data_train_reviews = 'ArtsReviews_train.txt'
  data_test_reviews = 'ArtsReviews_test.txt'
  data_train_ratings = 'ArtsRatings_train.txt'
  data_test_ratings = 'ArtsRatings_test.txt'
else:
  # Arts 5000
  data_all_reviews = 'Reduced_ArtsReviews_5000.txt'
  data_all_ratings = 'Reduced_ArtsRatings_5000.txt'
  data_train_reviews = 'ArtsReviews_5000_train.txt'
  data_test_reviews = 'ArtsReviews_5000_test.txt'
  data_train_ratings = 'ArtsRatings_5000_train.txt'
  data_test_ratings = 'ArtsRatings_5000_test.txt'

ArtsRatings_5000_test.txt   ArtsReviews_5000_train.txt	Reduced_ArtsRatings_5000.txt
ArtsRatings_5000_train.txt  glove.6B.100d.txt		Reduced_ArtsReviews_5000.txt
ArtsReviews_5000_test.txt   metamorphosis_clean.txt	review_polarity


##**1.fit the LSA model**

In [ ]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# from keras.preprocessing.text import Tokenizer
from pandas import DataFrame
from matplotlib import pyplot
import random
import nltk
import numpy
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import rand
from sklearn.metrics.pairwise import cosine_similarity
from numpy import argsort
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter

nltk.download('stopwords')

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# update counts
	vocab.update(tokens)

 # load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()

# split train and test
def random_sample(num1, num2):
    dataList = list(range(num1))
    TrainIndex = []
    for i in range(num2):
        randIndex = int(random.uniform(0,len(dataList)))
        TrainIndex.append(dataList[randIndex])
        del(dataList[randIndex])
    TestIndex = dataList
    return TrainIndex,TestIndex

# load doc into memory
def load_doc_lines(filename):
	 file = open(filename,'rt')
	 lines = list()
   # read all text as lines
	 while 1:

		 line = file.readline()
		 if not line:
		   break
		 pass
		 lines.append(line.strip("\n"))
   # close the file
	 file.close()
	 return lines

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()

	# convert to lower case
	tokens = [word.lower() for word in tokens]

 	# stemming of words
	from nltk.stem.porter import PorterStemmer
	porter = PorterStemmer()
	tokens = [porter.stem(word) for word in tokens]

	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

# load doc, clean and return line of tokens
def doc_to_line(doc):
	# load the doc
	# doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	# tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)

# load all docs in a directory
def process_docs(files):
	lines = list()
	# walk through all files in the folder
	for doc in files:
		# print(len(doc))
		line = doc_to_line(doc)
		# add to list
		lines.append(line)
	return lines

# prepare words encoding of docs
def prepare_data(train_docs, mode, vocab):
	# encode training data set
	vectorizer = CountVectorizer(vocabulary=vocab)
	transformer = TfidfTransformer(norm='l2')
	Xtrain = transformer.fit_transform(vectorizer.fit_transform(train_docs))
	return Xtrain

# load the dataset #########################################
ArRe_train_lines = load_doc_lines(data_path + data_all_reviews)

train_docs = process_docs(ArRe_train_lines)

# generate vocabulary
vocab = []
for ll in train_docs:
  tt = ll.split()
  for ww in tt:
    if ww not in vocab:
      vocab.append(ww)
# print(vocab)
# print(len(vocab))

Xtrain = prepare_data(train_docs, 'tfidf', vocab)

trunc_SVD_model = TruncatedSVD(n_components=50)
approx_Xtrain = trunc_SVD_model.fit_transform(Xtrain)
print("Approximated Xtrain shape: " + str(approx_Xtrain.shape))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Approximated Xtrain shape: (5000, 50)


##**2.retrieval**

In [ ]:
# preprocess query
def preprocess_query(review, mode, vocab):
	# clean
	tokens = clean_doc(review)
	# convert to line
	line = ' '.join(tokens)
	# encode
	vectorizer = CountVectorizer(vocabulary=vocab)
	transformer = TfidfTransformer(norm='l2')
	encoded = transformer.fit_transform(vectorizer.fit_transform([line]))
	return encoded

querys = ['I really enjoy these scissors!',
          'I hate the pen!'
]
for query in querys:
  # retrieval
  encoded_query = preprocess_query(query, 'tfidf', vocab)
  # print(encoded_query.shape)

  transformed_query = trunc_SVD_model.transform(encoded_query)
  # print("Transformed query: " + str(transformed_query))
  # print("Query shape: " + str(transformed_query.shape))

  # print(type(approx_Xtrain))
  # print(approx_Xtrain)
  similarities = cosine_similarity(approx_Xtrain, transformed_query)
  # print("Similarities shape: " + str(similarities.shape))

  Top_n_reviews=10
  # indexes = np.argsort(similarities.flat)[-Top_n_reviews:]
  indexes = np.argsort(similarities.flat)[::-1]

  print('\n' + 'Query: ' + query)
  for i in range(Top_n_reviews):
    print("Top " + str(i+1) + ' result:')
    print("Reviews ID: " + str(indexes[i]))
    print(ArRe_train_lines[indexes[i]])
    # print("similarities: " + str(similarities.flat[indexes[i]]))




Query: I really enjoy these scissors!
Top 1 result:
Reviews ID: 4332
I really don't like. The price is really cheap, but it really worth &#34;every penny&#34;. So be prepare what you pay is what you gonna get!
Top 2 result:
Reviews ID: 699
If you are into serious hobbies, or anything that requires really high quality wood, I would not recommend this bag at all.It worked okay for what I needed, as I was just cutting up some wood to practice with, but for actual use, it's not really ideal.There were a lot of really large chunks of wood, that you really couldn't do a lot of stuff with. If you're looking for a lot of thin slices, this really isn't your bad. Additionally, I know that balsa wood is generally fragile, but some of the thin strips had already broken by the time I received my package.Overall, this wood wasn't really that high quality. It was very rough, and I've seen a lot better for a little bit higher prices.
Top 3 result:
Reviews ID: 1986
I haven't really used it yet, but I 